# 🎓 Advanced Prompt Engineering with Azure OpenAI & GPT-4.1  
  
> In this hands-on interactive notebook, you'll learn and apply best practices for advanced prompt engineering using **GPT-4.1** via the **Azure OpenAI** platform.  
  
---  
  
## 1️⃣ Setup: Azure OpenAI Client  
  
We'll load environment variables from a local file and create our Azure OpenAI client.  

In [1]:
# 1️⃣ Setup cell: Import libraries and load environment variables  
  
import os  
from openai import AzureOpenAI  
from dotenv import load_dotenv  
  
# Load Azure credentials from a .env file (you should have credentials.env in your directory)  
load_dotenv("credentials.env")  
  
# Create Azure OpenAI client  
client = AzureOpenAI(  
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-01-01-preview",  
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")  
)  
  
print("✅ Azure OpenAI client initialized.")  

✅ Azure OpenAI client initialized.


## 2️⃣ Your First Prompt: Hello, GPT-4.1!  
  
Let's run a *simple chat completion* to test the setup and confirm our credentials.  
  
**Instructions:** Fill in your deployment name as configured in Azure OpenAI Portal (replace `'YOUR_DEPLOYMENT_NAME'`).  

In [2]:
# 2️⃣ First prompt cell: Basic chat completion  
  
deployment_name = "gpt-4.1"  # <-- CHANGE THIS!  
  
response = client.chat.completions.create(  
    model=deployment_name,  
    messages = [  
        {"role": "system", "content": "You are a helpful assistant."},  
        {"role": "user", "content": "Hello! What can you do?"}  
    ]  
)  
  
print(response.choices[0].message.content)  

Hello! I can help with a wide range of tasks, such as:

- Answering questions on many topics (science, history, technology, etc.)
- Assisting with writing (essays, emails, stories, resumes, etc.)
- Summarizing or explaining complex information
- Creating study guides or helping with homework
- Brainstorming ideas (projects, gifts, content, etc.)
- Providing coding help or debugging code
- Conversational practice or role-playing scenarios
- Giving recommendations (books, movies, music, etc.)
- Helping structure plans (travel, events, schedules)
- Offering advice or resources on learning new skills

If you have something specific in mind, just ask! What would you like help with today?


---  
## 3️⃣ Advanced: Chain of Thought Prompting  
  
Instruct the model to show its reasoning step by step. This is called "chain of thought" (CoT) and is **very effective** with GPT-4.1.  
  
Try solving a math riddle or similar question.  

In [3]:
# 3️⃣ Chain-of-thought demo cell  
  
question = "If Alice has twice as many apples as Bob, and together they have 12, how many apples does Alice have?"  
  
response = client.chat.completions.create(  
    model=deployment_name,  
    messages = [  
        {"role": "system", "content": "You are an expert math tutor. Show your reasoning step by step before giving an answer."},  
        {"role": "user", "content": question}  
    ]  
)  
print(response.choices[0].message.content)  

Let's define variables:

- Let \( x \) be the number of apples Bob has.
- Since Alice has twice as many as Bob, Alice has \( 2x \) apples.
- Together, they have 12 apples.

Set up the equation:
\[
x + 2x = 12
\]

Simplify:
\[
3x = 12
\]

Solve for \( x \):
\[
x = \frac{12}{3} = 4
\]

Bob has 4 apples. Alice has \( 2x = 2 \times 4 = 8 \) apples.

**Answer:**
Alice has **8 apples**.


---  
## 4️⃣: Building Instruction-Heavy, Agentic Prompts  
  
GPT-4.1 follows instructions *literally* and *precisely*. Let's see how adding rules changes the response.  

In [4]:
# 4️⃣ Instruction-heavy prompt  
  
prompt = """  
You are a goal-oriented agent. Always:  
- Greet the user.  
- Explain your reasoning step by step before your answer.  
- If you don't know, say so directly.  
- Be concise.  
"""  
  
question = "What is the capital of Australia, and explain why?"  
  
response = client.chat.completions.create(  
    model=deployment_name,  
    messages = [  
        {"role": "system", "content": prompt},  
        {"role": "user", "content": question}  
    ]  
)  
print(response.choices[0].message.content)  

Hello! Let’s work through your question step by step.

First, you asked for the capital of Australia. Many people often think of Sydney or Melbourne, as they are the country’s largest cities. However, neither of these is the capital.

The reason is historical: when Australia became a federation in 1901, Sydney and Melbourne were rival cities. To resolve this, the government decided to create a new city—Canberra—and designated it as the capital in 1913. Its location was chosen as a compromise; it's situated roughly halfway between Sydney and Melbourne.

In summary: The capital of Australia is Canberra, chosen as a political compromise between Sydney and Melbourne.


---  
  
## 5️⃣ Agentic Workflows: Planning, Persistence & Tool Use  
  
GPT-4.1 excels when treated as an agent with clear multi-step objectives.  
In agentic prompts, *always include*:  
- **Persistence**: Instruct the model to keep going until the problem is solved  
- **Tool use**: Be explicit about tool access and not making up answers  
- **Planning/Reflection**: Tell the model to plan before and after each action  
  
Below is a system prompt pattern using all three, for coding, support, or research agents.  

In [5]:
# 5️⃣ Example Agentic System Prompt (for coding agent)  
  
SYS_PROMPT_AGENT = """  
You are an autonomous agent helping a developer fix code.  
- Continue working step by step until the issue is truly solved.  
- Only stop if you are certain the problem is fully resolved.  
- Use your file/tools to gather context – NEVER guess.  
- For every function/tool call, PLAN before calling it and REFLECT after you get the result—describe your plan and outcome each time.  
"""  
  
user_issue = "The function process_data throws a TypeError in some cases. Find and fix the bug."  
  
response = client.chat.completions.create(  
    model="gpt-4.1",  # <-- CHANGE THIS!
    messages = [  
        {"role": "system", "content": SYS_PROMPT_AGENT},  
        {"role": "user", "content": user_issue}  
    ]  
)  
  
print(response.choices[0].message.content)  

To help you debug the `process_data` function, I’ll need to see its definition and how it’s used.  

**Plan:**  
- I will search the codebase for the definition of the `process_data` function.  
- I will also look for any usages/calls of `process_data` to see the context in which the error occurs.

Let me search for `process_data` in your project.


---  
  
## 6️⃣ Best Practices for Tool Use  
  
When connecting GPT-4.1 to external tools/functions:  
- **Always use the tool/function schema, not in-prompt descriptions**  
- Clear names and parameter descriptions  
- Place tool usage examples in an `# Examples` section of your prompt  
  
The example cell below uses a (simulated) lookup tool—adapt for your API.  

In [11]:
import json  
  
# 6️⃣ Correct Tool schema for Azure OpenAI, including the full tool call cycle  
  
tools = [{  
    "type": "function",  
    "function": {  
        "name": "lookup_product",  
        "description": "Finds product info by product_id.",  
        "parameters": {  
            "type": "object",  
            "properties": {  
                "product_id": {"type": "string", "description": "Unique product identifier."}  
            },  
            "required": ["product_id"],  
        },  
    }  
}]  
  
system = (  
    "You are a product expert. Never guess product details: "  
    "call the lookup_product tool whenever you need information you don't see."  
    "\n# Examples"  
    "\nUser: What are the specs for product X123?"  
    "\nAssistant: I'll use the lookup_product tool for product X123."  
)  
user = "What is the warranty period for Y789?"  
  
# Step 1: Assistant receives question and calls the tool  
response = client.chat.completions.create(  
    model="gpt-4.1",  # or use your string e.g. "gpt-4.1"  
    messages=[  
        {"role": "system", "content": system},  
        {"role": "user", "content": user}  
    ],  
    tools=tools,  
    tool_choice="auto"  
)  
  
assistant_msg = response.choices[0].message  
  
# Step 2: Detect tool call and simulate tool's result  
if assistant_msg.tool_calls:  
    tool_call = assistant_msg.tool_calls[0]  
    args = json.loads(tool_call.function.arguments)  
    product_id = args["product_id"]  
  
    # Simulated "database" lookup  
    tool_result = f"Warranty for {product_id} is 24 months."  
  
    # Step 3: Return result as a tool message, then get final assistant response  
    followup = client.chat.completions.create(  
        model="gpt-4.1",  # <-- CHANGE THIS!
        messages=[  
            {"role": "system", "content": system},  
            {"role": "user", "content": user},  
            {  
                "role": "assistant",  
                "content": None,  
                "tool_calls": [tool_call]  
            },  
            {  
                "role": "tool",  
                "tool_call_id": tool_call.id,  
                "name": "lookup_product",  
                "content": tool_result  
            },  
        ],  
        tools=tools  
    )  
    print(followup.choices[0].message.content)  
else:  
    print(assistant_msg.content)  

The warranty period for Y789 is 24 months.


---  
  
## 7️⃣ Long Context: Handling Large Inputs  
  
GPT-4.1 supports up to **1 million tokens** in context. Strong structuring and clear delimiting improves retrieval and relevance when working with many files, docs, or data.  
  
**Best practices:**  
- Use Markdown or XML/HTML tags to delimit documents.  
- Repeat key instructions before and after large context blocks.  
- Tell the model whether to answer with external context only, or to combine it with its own knowledge.  
  
Below is an example pattern using XML for document injection with clear instructions.  
  

In [12]:
system_longctx = (  
    "You are a document retrieval assistant. Only answer using the provided document context."  
    "\nIf answer cannot be found, say 'I don't have the information needed to answer that.'"  
    "\n# External Context START\n"  
    "<doc id=1 title='Warranty Policy'>Product Y789 carries a 24-month warranty from purchase date.</doc>\n"  
    "<doc id=2 title='Returns Policy'>Returns accepted within 30 days of purchase.</doc>\n"  
    "# External Context END\n"  
    "Restate your instructions: only use the provided documents!"  
)  
user = "What is the warranty for Y789?"  
  
response = client.chat.completions.create(  
    model="gpt-4.1",  # <-- CHANGE THIS!
    messages=[  
        {"role": "system", "content": system_longctx},  
        {"role": "user", "content": user},  
    ]  
)  
print(response.choices[0].message.content)  

Product Y789 carries a 24-month warranty from purchase date.


---  
  
## 8️⃣ Inducing Explicit Planning (Chain of Thought)  
  
Prompting the model to **plan or explain step by step** can improve complex outputs.  
  
**When to use:**    
- Multi-hop document answers  
- Code debugging/patching  
- Tasks needing structured reasoning  
  
**Pattern:**  
List reasoning/investigation steps, and prompt the model to output them.  
  

In [13]:
system_cot = (  
    "You are a senior support engineer."  
    "\nFirst, break down the user's question. Second, review possible relevant documents. Third, explain your reasoning step by step before answering."  
)  
  
user = (  
    "Does Y789 qualify for free extended support if it's out of warranty but only by a week? Use policy documents if available."  
)  
  
response = client.chat.completions.create(  
    model="gpt-4.1",  # <-- CHANGE THIS!
    messages=[  
        {"role": "system", "content": system_cot},  
        {"role": "user", "content": user},  
    ]  
)  
print(response.choices[0].message.content)  

Let’s break down your question:

**Step 1: Break Down the Question**
- Product/System: Y789.
- Status: Out of warranty (just by one week).
- Query: Does it still qualify for "free extended support" despite being only recently out of warranty?
- Reference needed: Policy documents regarding extended/free support after warranty expiration.

**Step 2: Review Possible Relevant Documents**
Typical support policies clarify:
- Standard warranty coverage period.
- Availability and conditions for extended support (whether free or paid).
- Grace periods, if any, after warranty expiration.

You’d want to review:
- The official warranty and support policy for product Y789.
- Any documents or web pages mentioning a “grace period” or exceptions.
- The definition and terms of “free extended support” specific to Y789.

**Step 3: Reasoning Step by Step**
1. **Warranty Status:** If Y789 is out of warranty—even if only by one week—it’s generally considered out of standard coverage.
2. **Free Extended Supp

---  
  
## 9️⃣ Diffs and Patch Generation  
  
GPT-4.1 excels at generating file/code diffs and producing precise patch outputs. Leveraging standardized formats improves reliability and makes integration with downstream tools (like `git apply` or patching libraries) much easier.  
  
**Best practices:**  
- Ask the model to generate diffs in a format compatible with Unix `patch` or common diff tools.  
- Provide the context (old file, new requirement/instruction) and ask for a Unix-style or unified diff.  
- If you want the model to only output the diff, **explicitly request “Output ONLY the diff, and nothing else.”**  
- For automated pipelines: check result formatting and wrap output in code blocks.  
  
---  
  
#### Example: Code Patch with Unified Diff  
  

In [14]:
system_patch = (  
    "You are a patch generation tool. Only output valid unified diffs in code blocks.\n"  
    "When given an old file and a description of a required change, generate the unified diff.\n"  
    "Do NOT add any explanations or extra text.\n"  
    "\n"  
    "Example input:\n"  
    "[Old file]:\n"  
    "def add(a, b):\n"  
    "    return a + b\n"  
    "\n"  
    "[Instruction]: Add input type checks so only integers are allowed.\n"  
    "----\n"  
    "Example output:\n"  
    "```diff\n"  
    "--- original.py\n"  
    "+++ modified.py\n"  
    "@@ -1,3 +1,7 @@\n"  
    " def add(a, b):\n"  
    "+    if not (isinstance(a, int) and isinstance(b, int)):\n"  
    "+        raise TypeError('Inputs must be integers')\n"  
    "     return a + b\n"  
    "```"  
)  
  
user_patch = (  
    "[Old file]:\n"  
    "def calculate_price(base, tax):\n"  
    "    return base * (1 + tax)\n"  
    "\n"  
    "[Instruction]: Prevent negative base prices. Raise ValueError if base < 0."  
)  
  
response = client.chat.completions.create(  
    model="gpt-4.1",  # <-- CHANGE THIS!
    messages=[  
        {"role": "system", "content": system_patch},  
        {"role": "user", "content": user_patch},  
    ]  
)  
print(response.choices[0].message.content)  

```diff
--- original.py
+++ modified.py
@@ -1,3 +1,6 @@
 def calculate_price(base, tax):
+    if base < 0:
+        raise ValueError("Base price cannot be negative")
     return base * (1 + tax)
```


---  
  
### ✅ Pro Tips:  
- For file patching tasks, always clearly delimit old content, instruction, and desired output format.  
- For multi-file or more complex changes, explicitly ask for a diff per file.  
- Use Markdown code blocks for reliable parsing and downstream processing.  
  
---  
  
## 📚 End of Notebook  
  
You now have a complete hands-on notebook covering all primary OpenAI GPT-4.1 prompt engineering patterns: agentic workflows, function calls, chain-of-thought, long context, formatting, and patch/diff generation!  
  
If you need full worked examples or want to add advanced topics (like tool chaining or RAG best practices), let me know!  